In [4]:
import pandas as pd

df = pd.read_csv("data/california_wildfires_final_data.tsv", sep="\t")

In [6]:
df.head()

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,917791044158185473,917791044158185473_0,informative,1.0000,informative,0.6766,other_relevant_information,1.0000,other_relevant_information,0.6766,NaN,NaN,RT @Gizmodo: Wildfires raging through Northern...,http://pbs.twimg.com/media/DLyi_WYVYAApwNg.jpg,data_image/california_wildfires/10_10_2017/917...
1,917791130590183424,917791130590183424_0,informative,1.0000,informative,0.6667,infrastructure_and_utility_damage,1.0000,affected_individuals,0.6667,NaN,NaN,PHOTOS: Deadly wildfires rage in California ht...,http://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg,data_image/california_wildfires/10_10_2017/917...
2,917791291823591425,917791291823591425_0,informative,0.6813,informative,1.0000,other_relevant_information,0.6813,infrastructure_and_utility_damage,1.0000,severe_damage,1.0,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaaZV4AAjT7x.jpg,data_image/california_wildfires/10_10_2017/917...
3,917791291823591425,917791291823591425_1,informative,0.6813,not_informative,1.0000,other_relevant_information,0.6813,not_humanitarian,1.0000,NaN,NaN,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaZXUMAABAEZ.jpg,data_image/california_wildfires/10_10_2017/917...
4,917792092100988929,917792092100988929_0,informative,0.6727,informative,0.6612,other_relevant_information,0.6727,infrastructure_and_utility_damage,0.6612,severe_damage,1.0,RT @TIME: California's raging wildfires as you...,http://pbs.twimg.com/media/DLwNe-NXUAE0XCw.jpg,data_image/california_wildfires/10_10_2017/917...


In [7]:
df = df[["tweet_id", "text_human", "image_human", "tweet_text"]]
df.head()

,tweet_id,text_human,image_human,tweet_text
0,917791044158185473,other_relevant_information,other_relevant_information,RT @Gizmodo: Wildfires raging through Northern...
1,917791130590183424,infrastructure_and_utility_damage,affected_individuals,PHOTOS: Deadly wildfires rage in California ht...
2,917791291823591425,other_relevant_information,infrastructure_and_utility_damage,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...
3,917791291823591425,other_relevant_information,not_humanitarian,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...
4,917792092100988929,other_relevant_information,infrastructure_and_utility_damage,RT @TIME: California's raging wildfires as you...
